In [97]:
# Include so results on different machines are (should be) the same.
import numpy as np
from numpy.random import seed
seed(1)

from tensorflow import set_random_seed
set_random_seed(2)

import random as rn
rn.seed(3)

In [98]:
!jupyter nbconvert --to script Keras_BagnallCharacter_SimpleRNN.ipynb

[NbConvertApp] Converting notebook Keras_BagnallCharacter_SimpleRNN.ipynb to script
[NbConvertApp] Writing 14441 bytes to Keras_BagnallCharacter_SimpleRNN.py


In [99]:
import glob, os, json, re, unicodedata
from bs4 import BeautifulSoup

load_verbose = 0
loaded_labels = []
loaded_text = []
presidents = [
    "Barack Obama",
    "Donald J. Trump",
    "Dwight D. Eisenhower",
    "Franklin D. Roosevelt",
    "George Bush",
    "George W. Bush",
    "Gerald R. Ford",
    "Harry S. Truman",
    "Herbert Hoover",
    "Jimmy Carter",
    "John F. Kennedy",
    "Lyndon B. Johnson",
    "Richard Nixon",
    "Ronald Reagan",
    "William J. Clinton"
]

labels = {}
for idx, name in enumerate(presidents):
    labels[name] = idx

# load raw text files straight in, no parsing
file_to_label = {
    "Obama": "Barack Obama",
    "Trump": "Donald J. Trump",
    "Eisenhower": "Dwight D. Eisenhower",
    "Roosevelt": "Franklin D. Roosevelt",
    "Bush": "George Bush",
    "WBush": "George W. Bush",
    "Ford": "Gerald R. Ford",
    "Truman": "Harry S. Truman",
    "Hoover": "Herbert Hoover",
    "Carter": "Jimmy Carter",
    "Kennedy": "John F. Kennedy",
    "Johnson": "Lyndon B. Johnson",
    "Nixon": "Richard Nixon",
    "Reagan": "Ronald Reagan",
    "Clinton": "William J. Clinton"
}

directory = "../data/processed/"
for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    if any(prefix in arr[0] for prefix in file_to_label.keys()):
        loaded_labels = loaded_labels + [labels[file_to_label[arr[0]]]]
        raw = open(filename).read().decode("UTF-8").encode("ascii","ignore")
        loaded_text = loaded_text + [raw] 


print "Loaded", len(loaded_text), "speeches for", len(set(loaded_labels)), "presidents."
# processed2 now contains files generated from unprocessed
directory = "../data/processed3/"
for filename in glob.glob(os.path.join(directory, '*.txt')):
    arr = filename.replace(directory,'').split("_")
    if any(prefix in arr[0] for prefix in file_to_label.keys()):
        loaded_labels = loaded_labels + [labels[file_to_label[arr[0]]]]
        raw = open(filename).read().decode("UTF-8").encode("ascii","ignore")
        loaded_text = loaded_text + [raw] 

print "Loaded", len(loaded_text), "speeches for", len(set(loaded_labels)), "presidents."

Loaded 30 speeches for 2 presidents.
Loaded 1747 speeches for 15 presidents.


In [100]:
#
# Bagnall 2015 text pre-processing
#
from string import maketrans
import re

chars_to_replace = "[]%!()>=*&_}+"
sub_chars = len(chars_to_replace) * " "
trantab = maketrans(chars_to_replace, sub_chars)
for x in range(0,len(loaded_text)):
    # "Various rare characters that seemed largely equivalent are mapped together..."
    loaded_text[x] = re.sub('`', '\'', loaded_text[x])
    loaded_text[x] = re.sub('--', '-', loaded_text[x])
    loaded_text[x] = re.sub('\n\n', '\n', loaded_text[x])
    # "...all digits in all languages are mapped to 7"
    loaded_text[x] = re.sub('[0-9]+', '7', loaded_text[x])
    # "...any character with a frequency lower than 1 in 10,000 is discarded."
    loaded_text[x] = loaded_text[x].translate(trantab)
    # "Runs of whitespace are collapsed into a single space."
    loaded_text[x] = re.sub(' +', ' ', loaded_text[x])

print "Replacements complete."

Replacements complete.


In [101]:
# Have a look at a scrubbed text excerpt
print loaded_text[10][:750]

Thank you. Thank you, everybody. I am thrilled to be here in the great state of Kentucky and the beautiful city of Louisville And this place is packed. There are a lot of people outside that arent getting in, but thats all right. We love them, too, right? We love them, too. 
Were in the heartland of America, and there is no place I would rather be than here with you, tonight. Our first Republican President, Abraham Lincoln, was born right here in Kentucky. Thats not bad. The legendary pioneer, Daniel Boone, helped settle the Kentucky frontier. And the great 7th century American statesman, Henry Clay, represented Kentucky in the United States Congress. Henry Clay believed in what he called the American system, and proposed tariffs to protect


In [102]:
#
# Join all speeches into one massive per president
#  for later processing
#
import numpy as np
from scipy import stats
from operator import itemgetter
from collections import defaultdict

compressed_text = [None]*(len(labels))
for key, value in sorted(labels.iteritems()):
    compressed_text[value] = ""
    for idx in range(0,len(loaded_text)):
        if (loaded_labels[idx] == value):
            compressed_text[value] = compressed_text[value] + loaded_text[idx] + " "
            
print "How many characters of text per president?"
for key, value in sorted(labels.iteritems()):
    print str(value).ljust(2), ":", key.ljust(20), "\t", len(compressed_text[value])

label_min_chars = len(min(compressed_text, key=len))
print "\nMinimum number of characters per president?"
print label_min_chars

How many characters of text per president?
0  : Barack Obama         	4869126
1  : Donald J. Trump      	555750
2  : Dwight D. Eisenhower 	3062523
3  : Franklin D. Roosevelt 	1741086
4  : George Bush          	1886633
5  : George W. Bush       	1772167
6  : Gerald R. Ford       	706711
7  : Harry S. Truman      	1988667
8  : Herbert Hoover       	778333
9  : Jimmy Carter         	1283640
10 : John F. Kennedy      	1337590
11 : Lyndon B. Johnson    	2318863
12 : Richard Nixon        	969424
13 : Ronald Reagan        	982788
14 : William J. Clinton   	1765554

Minimum number of characters per president?
555750


In [103]:
#
# Tokenize words into chars
#
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

# Tokenize into characters
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(compressed_text)
tokenized_text = tokenizer.texts_to_sequences(compressed_text)

# there's an oddity in the encoding for some reason where a len+1 character occurs
unique_chars = len(tokenizer.word_counts)+1

print "Unique char count:", unique_chars
print "\nChars w/ counts:"
print sorted(((v,k) for k,v in tokenizer.word_counts.iteritems()), reverse=True)

Unique char count: 68

Chars w/ counts:
[(4692463, ' '), (2550095, 'e'), (2117905, 't'), (1615001, 'o'), (1598810, 'a'), (1465361, 'n'), (1394749, 'i'), (1183917, 's'), (1147156, 'r'), (1059389, 'h'), (749147, 'l'), (699363, 'd'), (544092, 'u'), (529907, 'c'), (475112, 'm'), (415048, 'w'), (390669, 'g'), (385818, 'f'), (379109, 'p'), (371935, 'y'), (279440, 'b'), (262578, ','), (262036, '.'), (239101, 'v'), (175280, 'I'), (150340, 'k'), (95803, "'"), (74330, 'A'), (58657, 'T'), (58270, 'S'), (44149, '-'), (40497, '7'), (40258, 'W'), (35453, 'C'), (33702, 'N'), (32672, 'x'), (27565, 'M'), (25721, 'P'), (25335, 'j'), (25299, 'B'), (23864, 'E'), (21606, 'q'), (20219, 'R'), (19871, 'O'), (19463, 'G'), (17125, 'H'), (16203, 'U'), (15821, 'D'), (13768, 'F'), (12806, 'L'), (11786, 'z'), (10651, 'Y'), (10586, '"'), (9196, ';'), (8111, '?'), (8043, 'J'), (6308, ':'), (5273, 'V'), (4481, 'K'), (4083, '\n'), (3981, '$'), (1853, '/'), (1348, 'Q'), (446, 'Z'), (421, 'X'), (6, '#'), (4, '\\')]


In [146]:
#
# Split speeches into subsequences 
#
from collections import Counter

def splits(_list, _split_size):
    output_list = []
    for idx in range(0, len(_list), _split_size):
        if (idx + _split_size) <= len(_list):
            output_list.append(_list[idx:idx + _split_size])
    return output_list

max_seq_len = 50

# create new speech/label holders
split_text = []
split_labels = []

for idx in range(0, len(tokenized_text)):
    current_label = idx
    current_speech = tokenized_text[idx]#[:label_min_chars]
    current_splits = splits(current_speech, max_seq_len)
    split_text.extend(current_splits)
    split_labels.extend([current_label] * len(current_splits))

print "Subsequence total count; subsequence label total count:", len( split_text ), len( split_labels )
print "\nTotal characters:", len( split_text ) * max_seq_len

Subsequence total count; subsequence label total count: 1040747 1040747

Total characters: 26018675


In [147]:
# Have a look at a few split text excerpts
print split_text[10:15]
print split_labels[10:15]

[[2, 9, 5, 3, 7, 4, 6, 1, 10, 2, 1, 10, 5, 8, 1, 8, 10, 4, 16, 6, 1, 3, 10, 9, 4], [13, 17, 10, 4, 13, 3, 1, 3, 10, 7, 8, 1, 3, 9, 5, 6, 8, 7, 3, 7, 4, 6, 23, 60, 49], [4, 9, 3, 20, 31, 18, 4, 13, 9, 1, 28, 15, 2, 9, 7, 14, 5, 6, 8, 1, 10, 5, 24, 2, 1], [6, 4, 16, 1, 3, 5, 26, 2, 6, 1, 3, 10, 2, 1, 38, 9, 2, 8, 7, 12, 2, 6, 3, 7, 5], [11, 1, 4, 5, 3, 10, 23, 1, 29, 10, 2, 1, 16, 4, 9, 12, 8, 1, 10, 5, 24, 2, 1, 21, 2]]
[0, 0, 0, 0, 0]


In [148]:
#
# split amongst speaker samples, not the whole population of samples; shuffle if requested
#
import sklearn.utils

def split_test_train(input_text, input_labels, labels, train_pct=0.8, shuffle_p=True):
    train_text = []
    train_labels = []
    test_text = []
    test_labels = []

    for key, value in sorted(labels.iteritems()):
        # grab all values of a specific label
        subset_text = list(itemgetter(*[idx for idx, label in enumerate(input_labels) if label == value ])(input_text))
        subset_labels = list(itemgetter(*[idx for idx, label in enumerate(input_labels) if label == value ])(input_labels))

        cut_pos = int(train_pct * len(subset_text))
        train_text = train_text + subset_text[:cut_pos]
        train_labels = train_labels + subset_labels[:cut_pos]
        test_text = test_text + subset_text[cut_pos:]
        test_labels = test_labels + subset_labels[cut_pos:]

    if shuffle_p:
        test_text, test_labels = sklearn.utils.shuffle(test_text, test_labels)
        train_text, train_labels = sklearn.utils.shuffle(train_text, train_labels)

    return train_text, train_labels, test_text, test_labels

In [153]:
#
# Prep test/train
#
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight

# split data smartly
train_X, train_y, test_X, test_y = split_test_train(split_text, split_labels, 
                                                    labels, train_pct=0.8, shuffle_p=False)
print "Sample splits:\n Test = ", len(train_X), "\n Train = ", len(test_X)##

# compute class weights to account for imbalanced classes
y_weights = (class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)).tolist()
y_weights = dict(zip(sorted(labels.values()), y_weights))
print "\nClass weights:\n", y_weights

Sample splits:
 Test =  832592 
 Train =  208155

Class weights:
{0: 0.35623785930052454, 1: 3.1211276053381316, 2: 0.5663891156462585, 3: 0.9962690406959351, 4: 0.9194019302546435, 5: 0.9788060473536949, 6: 2.454503110167743, 7: 0.8722442223479372, 8: 2.2286249631949464, 9: 1.3513032752296557, 10: 1.2968116754668786, 11: 0.7480308523015691, 12: 1.7893660004298302, 13: 1.7650131433901466, 14: 0.9824616056309775}


In [154]:
# Have a look at a few of the split text excerpts; 
# likely the same classes based on the split point
print train_X[10:15]
print train_y[10:15]

[[2, 9, 5, 3, 7, 4, 6, 1, 10, 2, 1, 10, 5, 8, 1, 8, 10, 4, 16, 6, 1, 3, 10, 9, 4], [13, 17, 10, 4, 13, 3, 1, 3, 10, 7, 8, 1, 3, 9, 5, 6, 8, 7, 3, 7, 4, 6, 23, 60, 49], [4, 9, 3, 20, 31, 18, 4, 13, 9, 1, 28, 15, 2, 9, 7, 14, 5, 6, 8, 1, 10, 5, 24, 2, 1], [6, 4, 16, 1, 3, 5, 26, 2, 6, 1, 3, 10, 2, 1, 38, 9, 2, 8, 7, 12, 2, 6, 3, 7, 5], [11, 1, 4, 5, 3, 10, 23, 1, 29, 10, 2, 1, 16, 4, 9, 12, 8, 1, 10, 5, 24, 2, 1, 21, 2]]
[0, 0, 0, 0, 0]


In [155]:
#
# One-hot encoding classes & samples
#
from keras.utils import to_categorical

# one-hot encode classes
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

# one-hot encode samples
train_X = np.array(train_X)
orig_train_X_size=train_X.shape[0]
print "Encoding train_X with dimensions ", train_X.shape
train_X = to_categorical(train_X, num_classes=unique_chars)
print "...to ", train_X.shape
train_X = np.reshape(train_X,(orig_train_X_size,max_seq_len,unique_chars))
print "...and reshaping to ", train_X.shape

test_X = np.array(test_X)
orig_test_X_size=test_X.shape[0]
print "\nEncoding test_X with dimensions ", test_X.shape
test_X = to_categorical(test_X, num_classes=unique_chars)
print "...to ", test_X.shape
test_X = np.reshape(test_X,(orig_test_X_size,max_seq_len,unique_chars))
print "...and reshaping to ", test_X.shape

Encoding train_X with dimensions  (832592, 25)
...to  (20814800, 68)
...and reshaping to  (832592, 25, 68)

Encoding test_X with dimensions  (208155, 25)
...to  (5203875, 68)
...and reshaping to  (208155, 25, 68)


In [156]:
# Have a again look at a few of the split and encoded text excerpts; 
# both arrays should be one-hot encoded.
print train_X[10:11]
print train_y[10:11]

[[[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
  [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
    0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [157]:
# custom activation from Bagnall 2015
#  we were never able to get this to work; either nan'ed or never converged
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects

def ReSQRT(x):
    cond = tf.less_equal(x, 0.0)
    return tf.where(cond, x * 0.0, tf.sqrt(x+1)-1)

get_custom_objects().update({'ReSQRT': ReSQRT})

Bagnall proposes that the following possible values contribute to the success of the model:

| meta-parameter                  	| typical value                      	|
|---------------------------------	|------------------------------------	|
| initial adagrad learning scale  	| 0.1, 0.14, 0.2, 0.3                	|
| initial leakage between classes 	| 1/4N to 5/N                        	|
| leakage decay (per sub-epoch)   	| 0.67 to 0.9                        	|
| hidden neurons                  	| 79, 99, 119, 139                   	|
| presynaptic noise σ             	| 0, 0.1, 0.2, 0.3, 0.5              	|
| sub-epochs                      	| 6 to 36                            	|
| text direction                  	| forward or backward                	|
| text handling                   	| sequential, concatenated, balanced 	|
| initialisation                  	| gaussian, zero                     	|

In [161]:
##
## BASELINE
##
from keras.layers import Input, Dense, SimpleRNN, Bidirectional, Dropout
from keras.callbacks import ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adagrad, adam
from keras.models import Model
from keras.utils import plot_model

# define operating vars
batch_size = 50
epochs = 100

# define optimizer
optimizer = Adagrad(lr=0.001)

# define any callbacks
reduce_lr = ReduceLROnPlateau(monitor='categorical_accuracy', factor=0.5,
              patience=1, verbose=1)
csv_logger = CSVLogger('Keras_BagnallCharacter_SimpleRNN.log')

# assemble & compile model
main_input = Input(shape=(max_seq_len,unique_chars,))
rnn = Bidirectional(SimpleRNN(units=100,activation="tanh"))(main_input)
drop = Dropout(0.2)(rnn)
main_output = Dense(len(labels),activation='softmax',kernel_initializer='random_uniform')(drop)
model = Model(inputs=[main_input], outputs=[main_output])

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['categorical_accuracy'])
plot_model(model, to_file='Keras_BagnallCharacter_SimpleRNN.png', show_shapes=True, show_layer_names=True)
print(model.summary())


# train
model.fit([np.array(train_X)],
          [np.array(train_y)],
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          class_weight = y_weights,
          callbacks=[reduce_lr, csv_logger],
          verbose=1)

model.save('Keras_BagnallCharacter_SimpleRNN.h5')  
del model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 25, 68)            0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 200)               33800     
_________________________________________________________________
dense_13 (Dense)             (None, 15)                3015      
Total params: 36,815
Trainable params: 36,815
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
143650/832592 [====>.........................] - ETA: 432s - loss: 2.7122 - categorical_accuracy: 0.0650

KeyboardInterrupt: 

In [ ]:
# Load computed model
from keras.models import load_model
# returns a compiled model identical to the one trained
model = load_model('Keras_BagnallCharacter_SimpleRNN.h5')

In [ ]:
from sklearn import metrics

# Evaluate performance
print "Evaluating test data..."
loss_and_metrics = model.evaluate(test_X, test_y)
print model.metrics_names
print loss_and_metrics

# Make some predictions
print "\nPredicting using test data..."
pred_y = model.predict(test_X, batch_size=batch_size, verbose=1)
pred_y_collapsed = np.argmax(pred_y, axis=1)
test_y_collapsed = np.argmax(test_y, axis=1)
print "\n\nDone prediction."

print "\nAUC = ", metrics.roc_auc_score(test_y, pred_y)

In [ ]:
# Plot confusion matrix
#   from scikit-learn examples @
#   http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html 
%matplotlib inline
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
        print(np.sum(cm,axis=0))

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(test_y_collapsed, pred_y_collapsed)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=(sorted(labels, key=labels.get)),
                      title='Confusion matrix, without normalization')

#Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(np.round(cnf_matrix,2), classes=(sorted(labels, key=labels.get)), normalize=True,
                      title='Normalized confusion matrix')

plt.show()